In [2]:
import numpy as np
from skimage import io, color, exposure, transform
from skimage.color import rgb2gray
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split  #it came from update scikit learn. https://stackoverflow.com/questions/40704484/importerror-no-module-named-model-selection
import os
import glob
import h5py
import keras

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, model_from_json
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D,Conv1D
from keras.layers.pooling import MaxPooling2D

from keras.layers.convolutional import Convolution2D, MaxPooling2D

from keras.optimizers import SGD
from keras.utils import np_utils
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras import backend as K
K.set_image_data_format('channels_first')

from matplotlib import pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
from matplotlib.pylab import rcParams

#import keras

NUM_CLASSES = 19 # change it with respect to the desired class
IMG_SIZE = 197 # change it if it desired
IMG_depth = 3 # for RGB 3, for B&W it will be 1

from keras.applications import VGG19, MobileNet, ResNet50

In [3]:
import datetime
current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print("current time:", current_time)

current time: 2019-10-29 11:26:48


In [4]:
def preprocess_img(img):
    # Histogram normalization in y
    hsv = color.rgb2hsv(img)
    hsv[:,:,2] = exposure.equalize_hist(hsv[:,:,2])
    img = color.hsv2rgb(hsv)

    # central scrop
    min_side = min(img.shape[:-1])
    centre = img.shape[0]//2, img.shape[1]//2
    img = img[centre[0]-min_side//2:centre[0]+min_side//2,centre[1]-min_side//2:centre[1]+min_side//2,:]
#    img = rgb2gray(img)

    # rescale to standard size
    img = transform.resize(img, (IMG_SIZE, IMG_SIZE))

    # roll color axis to axis 0
    img = np.rollaxis(img,-1) #this line is doing the channel first operation

    return img

def get_class(img_path):
    return int(img_path.split('/')[-2])
#     return str(img_path.split('/')[-2]) # returning the folder name. If use -1 that means image name. consider the img_path.

In [5]:
path = '/home/atif/machine_learning_stuff/model_file_keras/'

In [6]:
# ResNet50_base = ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape=(IMG_depth,IMG_SIZE,IMG_SIZE))

In [ ]:
imgs = []
labels = []
root_dir = '/home/atif/machine_learning_stuff/ml_image/train_image_classification_better_resolution/crop/'
#path='/home/atif/training_by_several_learning_process/flower_photos/00000/'

#all_img_paths = glob.glob(path+ '5547758_eea9edfd54_n_000.jpg')

all_img_paths = glob.glob(os.path.join(root_dir, '*/*')) #I have done the training with .png format image. If another type of image will come 
                                                                                    #them .png will be changed by that extension
np.random.shuffle(all_img_paths)
for img_path in all_img_paths:
    try:
        img = preprocess_img(io.imread(img_path))
        label = get_class(img_path)
        imgs.append(img)
        labels.append(label)

        if len(imgs)%1200 == 0: print("Processed {}/{}".format(len(imgs), len(all_img_paths)))
            #print("get it 2")
    except (IOError, OSError):
        print('missed', img_path)
        pass

Processed 1200/61560
Processed 2400/61560
Processed 3600/61560
Processed 4800/61560
Processed 6000/61560
Processed 7200/61560
Processed 8400/61560
Processed 9600/61560
Processed 10800/61560
Processed 12000/61560
Processed 13200/61560
Processed 14400/61560
Processed 15600/61560
Processed 16800/61560
Processed 18000/61560
Processed 19200/61560
Processed 20400/61560
Processed 21600/61560
Processed 22800/61560
Processed 24000/61560
Processed 25200/61560
Processed 26400/61560
Processed 27600/61560
Processed 28800/61560
Processed 30000/61560
Processed 31200/61560
Processed 32400/61560
Processed 33600/61560
Processed 34800/61560
Processed 36000/61560
Processed 37200/61560
Processed 38400/61560
Processed 39600/61560
Processed 40800/61560
Processed 42000/61560
Processed 43200/61560
Processed 44400/61560
Processed 45600/61560
Processed 46800/61560
Processed 48000/61560
Processed 49200/61560
Processed 50400/61560
Processed 51600/61560


In [ ]:
X = np.array(imgs, dtype='float32') #Keeping the image as an array
X = X.reshape(len(imgs),IMG_depth,IMG_SIZE,IMG_SIZE) # write (IMG_SIZE,IMG_SIZE,1 if you want channel last; 1= grayscale;3=RGB)
# Y = np.eye(NUM_CLASSES, dtype='uint8')[labels]
Y = keras.utils.to_categorical(labels, NUM_CLASSES)

print('X shape: ', X.shape,' type: ',type(X))
print('Y shape: ', Y.shape,' type: ',type(Y))

In [ ]:
conv_base_1 = VGG19(weights='imagenet', include_top=True)

In [ ]:
conv_base_1.summary()

In [7]:
# conv_base = VGG19(weights='imagenet', include_top=False, input_shape=(3,48,48))

# pre_trained_weight = VGG19(weights=path+'vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',include_top=False, input_shape=(IMG_depth,IMG_SIZE,IMG_SIZE))

pre_trained_weight = ResNet50(weights=path+'resnet/'+'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',include_top=False, input_shape=(IMG_depth,IMG_SIZE,IMG_SIZE))

In [ ]:
# # if want to download weight file

# from keras.applications import InceptionResNetV2

# conv_base = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(48,48,3))

In [ ]:
# from keras.applications import InceptionResNetV2

# conv_base = InceptionResNetV2(weights='/home/atif/machine_learning_stuff/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5',
#                               include_top=False, input_shape=(3,150,150))

In [8]:
for layer in pre_trained_weight.layers[:-4]:
    layer.trainable = False
# Check the trainable status of the individual layers
for layer in pre_trained_weight.layers:
    print(layer, layer.trainable)

<keras.engine.topology.InputLayer object at 0x7f08bf0def28> False
<keras.layers.convolutional.Conv2D object at 0x7f08bf0dec18> False
<keras.layers.normalization.BatchNormalization object at 0x7f08bf0de710> False
<keras.layers.core.Activation object at 0x7f08bfb156a0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f08bfb28c18> False
<keras.layers.convolutional.Conv2D object at 0x7f08bf0d3978> False
<keras.layers.normalization.BatchNormalization object at 0x7f08bfb34b00> False
<keras.layers.core.Activation object at 0x7f08bf672630> False
<keras.layers.convolutional.Conv2D object at 0x7f08bf4beda0> False
<keras.layers.normalization.BatchNormalization object at 0x7f08bf206e10> False
<keras.layers.core.Activation object at 0x7f08bf8ecf98> False
<keras.layers.convolutional.Conv2D object at 0x7f08bf8503c8> False
<keras.layers.convolutional.Conv2D object at 0x7f08bf1cf898> False
<keras.layers.normalization.BatchNormalization object at 0x7f08bf5e2710> False
<keras.layers.normalization.Ba

In [9]:
# pre_trained_weight.summary()

In [10]:
def cnn_model():
    model = Sequential()
    
    model.add(pre_trained_weight)
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    return model

In [11]:
model = cnn_model()

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048, 1, 1)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              8392704   
_________________________________________________________________
dropout_1 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 19)                77843     
Total params: 32,058,259
Trainable params: 8,474,643
Non-trainable params: 23,583,616
_________________________________________________________________


In [ ]:
# print('Number of trainable weights before freezing the conv base:', len(model.trainable_weights))
# conv_base.trainable = False
# print('Number of trainable weights after freezing the conv base:', len(model.trainable_weights))

In [ ]:
lr = 0.01
sgd = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
          optimizer=sgd,
          metrics=['accuracy'])

In [ ]:
def lr_schedule(epoch):
    return lr * (0.1 ** int(epoch / 10))

batch_size = 32
epochs = 50
do_train_model=model.fit(X, Y,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2,verbose=2,
          #np.resize(img, (-1, <image shape>)
          callbacks=[LearningRateScheduler(lr_schedule),ModelCheckpoint(path+str(current_time)+'_vgg19_epoch_'+str(epochs)+'.h5', save_best_only=True)])

In [1]:
import numpy as np

In [3]:
4%2

0

In [7]:
x = 57720

for i in range(1,x+1):
    if x%i == 0:
        print(i)

1
2
3
4
5
6
8
10
12
13
15
20
24
26
30
37
39
40
52
60
65
74
78
104
111
120
130
148
156
185
195
222
260
296
312
370
390
444
481
520
555
740
780
888
962
1110
1443
1480
1560
1924
2220
2405
2886
3848
4440
4810
5772
7215
9620
11544
14430
19240
28860
57720
